In [16]:
import numpy as np
import itertools as it
import pandas as ps

In [17]:
# "global" constants
rels = ["friend", "enemy"]

all_games = dict(
    pd = [[8, 8], # Result 1
          [0,12], # 2
          [12,0], # 3
          [4, 4]] # 4
    ,
    threat = [[12,6],
              [6,12],
              [6,0],
              [0,6]]
    ,
    disjunctive = [[12,12],
                   [12,12],
                   [12,12],
                   [0,0]]
    ,
    coordination = [[12,12],
                  [ 0, 0],
                  [ 0, 0],
                  [12,12]]
    ,
    singleControl = [[6, 6],
                  [0, 6],
                  [6, 6],
                  [0, 6]]
)

test_game = all_games["coordination"]
# test_game

In [18]:
def quick_format(s):
    return np.around(s, decimals=3).tolist()

In [19]:
def switch_player(player):
    if player.lower() == "a":
        return "b";
    elif player.lower() == "b":
        return "a";
    else:
        raise ValueError("Was not a player 'a' or 'b'!")

In [20]:
def calculate_TU(result, w1, w2, gb, player):
    u1 = result[0]
    u2 = result[1]
    
    # alternate perspectives
    if player.lower() == "a":
        return (w1 * u1) + (gb * w2 * u2);
    elif player.lower() == "b":
        return (w1 * u2) + (gb * w2 * u1);
    else:
        raise ValueError("Was not a player 'a' or 'b'!")

In [47]:
def calculate_EU(u1, u2, p1=0.5, p2=0.5):
    return (u1 * p1) + (u2 * p2); # p: probs of choices

In [22]:
def calculate_probs(eu_choice, eu_other): # probability of a choice
    return np.exp(eu_choice) / (np.exp(eu_choice) + np.exp(eu_other))

In [222]:
def predict(all_tu, player):  # "innermost" call in recursion
    all_eu = [] # keeps default p=0.5 values - see calculate_EU
    if player.lower() == "a":
        all_eu.append(calculate_EU(all_tu[0], all_tu[1])) #a1
        all_eu.append(calculate_EU(all_tu[2], all_tu[3])) #a2
    elif player.lower() == "b":
        all_eu.append(calculate_EU(all_tu[0], all_tu[2])) #b1
        all_eu.append(calculate_EU(all_tu[1], all_tu[3])) #b2
    else:
        raise ValueError("Was not a player 'a' or 'b'!")
    
    probs = []
    probs.append(calculate_probs(all_eu[0], all_eu[1])) # a1 or b1
    probs.append(calculate_probs(all_eu[1], all_eu[0])) # a2 or b2
    
    return probs;

In [223]:
def predict_recursive(game, w1, w2, gb, player): # player is current level i.e. A (me) simulating B -> player=B
    all_tu = [] # array of total utilities for player in each result
    for result in game:
        all_tu.append(calculate_TU(result, w1, w2, gb, player))
    
    p = predict(all_tu, switch_player(player)) # probs based on p1 = p2 = 0.5
    
    all_eu = []
    if player.lower() == "a":
        all_eu.append(calculate_EU(all_tu[0], all_tu[1], p[0], p[1])) #a1
        all_eu.append(calculate_EU(all_tu[2], all_tu[3], p[0], p[1])) #a2
    elif player.lower() == "b":
        all_eu.append(calculate_EU(all_tu[0], all_tu[2], p[0], p[1])) #b1
        all_eu.append(calculate_EU(all_tu[1], all_tu[3], p[0], p[1])) #b2
    
    probs = []
    probs.append(calculate_probs(all_eu[0], all_eu[1])) # a1 or b1
    probs.append(calculate_probs(all_eu[1], all_eu[0])) # a2 or b2
    
    return probs;

In [300]:
def predict2(game, w1, w2, gb, player, r):
    all_tu = [] # array of total utilities for player in each result
    for result in game:
        all_tu.append(calculate_TU(result, w1, w2, gb, player))
        
    all_eu = []  # uses 
    all_eu.append(all_tu[0] * r[0]) #a1b1
    all_eu.append(all_tu[1] * r[1]) #a1b2
    all_eu.append(all_tu[2] * r[2]) #a2b1
    all_eu.append(all_tu[3] * r[3]) #a2b2
    
    probs = []
    if player.lower() == "a":
        probs.append(calculate_probs(all_eu[0], all_eu[2])) # b1a1
        probs.append(calculate_probs(all_eu[1], all_eu[3])) # b2a1
        probs.append(calculate_probs(all_eu[2], all_eu[0])) # b1a2
        probs.append(calculate_probs(all_eu[3], all_eu[1])) # b2a2
    elif player.lower() == "b":
        probs.append(calculate_probs(all_eu[0], all_eu[1])) # a1b1
        probs.append(calculate_probs(all_eu[1], all_eu[0])) # a1b2
        probs.append(calculate_probs(all_eu[2], all_eu[3])) # a2b1
        probs.append(calculate_probs(all_eu[3], all_eu[2])) # a2b2
    
    return probs;

In [192]:
# returns probabilites of player's choice given friends P(c|f)
def get_choice_probs1(game, relationship, player1):
    # options for friends
    start = 0
    end = 50
    gb = 1
    if relationship=="enemies":
        gb = -1
    
    outcomes = []
    for i in range(start, end):
        w1 = i / 100 # counting by 0.01
        w2 = 1 - w1
        outcomes.append(predict_recursive(game, w1, w2, gb, player1)) # ex: returns simulation of player B
    
    final = np.mean(outcomes, axis=0)
#     print("Probability of choice {}1: {}".format(player.upper(), final[0]))
#     print("Probability of choice {}2: {}".format(player.upper(), final[1]))
    return final;

In [193]:
# returns probabilites of player's choice given friends P(c|f)
def get_choice_probs2(game, relationship, player2, r1):
    # options for friends
    start = 0
    end = 50
    gb = 1
    if relationship=="enemies":
        gb = -1
    
    outcomes = []
    for i in range(start, end):
        w1 = i / 100 # counting by 0.01
        w2 = 1 - w1
        outcomes.append(predict2(game, w1, w2, gb, player2, r1)) # ex: returns simulation of player B
    
    final = np.mean(outcomes, axis=0)
#     print("Probability of choice {}1: {}".format(player.upper(), final[0]))
#     print("Probability of choice {}2: {}".format(player.upper(), final[1]))
    return final; 

In [194]:
# get_outcome_probs(test_game)
get_choice_probs2(all_games["singleControl"], "friends", "b", [0.5, 0.5])

array([ 0.44973042,  0.05026958,  0.44973042,  0.05026958])

In [284]:
# Step 1: returns P(friends|A choice)
# a1, a2
def calc_friends_prob1(game, player1):
    p = get_choice_probs1(game, "friends", player1) * 0.5
    if player1 == "a":
        return [val for val in p for _ in (0, 1)]
    
    return p.tolist() * 2

In [303]:
# Step 2: returns P(friends|A then B choices)
# a1b1, a1b2, a2b1, a2b2
def calc_friends_prob2(game, player1):
    r1 = np.array(calc_friends_prob1(game, player1))
    r2 = get_choice_probs2(game, "friends", switch_player(player1), r1)
    
    r = r1 * r2 * 0.5

    return r.tolist() # results are proportional to actual probabilities
# FIXME: normalize by relationships f+e=1

In [288]:
# Step 1: returns P(enemies|A choice)
# a1, a2
def calc_enemies_prob1(game, player1):
    p = get_choice_probs1(game, "enemies", player1) * 0.5
    if player1 == "a":
        return [val for val in p for _ in (0, 1)]
    
    return p.tolist() * 2

In [304]:
# Step 2: returns P(friends|A then B choices)
# a1b1, a1b2, a2b1, a2b2
def calc_enemies_prob2(game, player1):
    r1 = np.array(calc_enemies_prob1(game, player1))
    r2 = get_choice_probs2(game, "enemies", switch_player(player1), r1)
    
    r = r1 * r2 * 0.5

    return r.tolist() # results are proportional to actual probabilities

In [305]:
def normalize(probs1, probs2):
    if len(probs1) != len(probs2):
        raise ValueError("Can't normalize!")
    
    total = 0
    ret_res = []

    for i in range(len(probs1)):
        total = probs1[i] + probs2[i]
        ret_res.append(probs1[i]/total)
    
    return ret_res;

In [261]:
def get_predictions(game, p1, stage):
    if stage == 1:
        return normalize(calc_friends_prob1(all_games[game], p1),
                         calc_enemies_prob1(all_games[game], p1))
    
    return normalize(calc_friends_prob2(all_games[game], p1),
                     calc_enemies_prob2(all_games[game], p1))

In [323]:
players = ["a","b"]
stages = [1, 2]

def generate_game_predictions():
    mp = []
    current = []
    for game in all_games.keys():
        for o in range(4): # 4 cells per game
            current = []
            current.append(game + str(o))
            for s in stages:
                for p in players:
                    current.append(p)
                    current.append("rangeProb" + str(s))
                    current.append(get_predictions(game, p, s)[o])
                    mp.append(current)
                    current = current[:1]

    predictions = ps.DataFrame(data=mp, columns=["game","firstPlayer","phase","rating"])
    remove_these = ["pd2","disjunctive1","disjunctive2","coordination2","coordination3",
                   "singleControl2","singleControl3","threat2","pd1"]
    predictions = predictions.loc[~predictions['game'].isin(remove_these)]
    predictions['firstPlayer'] = predictions['firstPlayer'].map({'a': 'row', 'b': 'column'})
    
#     predictions.to_csv("modelPredictions.csv",index=False)
    return predictions.reset_index(drop=True);

In [324]:
# test = generate_game_predictions()
# test

,game,firstPlayer,phase,rating
0,threat0,row,rangeProb1,0.900528
1,threat0,column,rangeProb1,0.997229
2,threat0,row,rangeProb2,0.944718
3,threat0,column,rangeProb2,0.998519
4,threat1,row,rangeProb1,0.900528
5,threat1,column,rangeProb1,0.099472
6,threat1,row,rangeProb2,0.766305
7,threat1,column,rangeProb2,0.233695
8,threat3,row,rangeProb1,0.002771
9,threat3,column,rangeProb1,0.099472


In [325]:
# test.to_csv("modelData.csv", index=False)